In [47]:
import cv2
import glob
import os
from subprocess import call
import time

# Augment data set

As described [here](http://docs.opencv.org/2.4/doc/user_guide/ug_traincascade.html#creating-training-set-from-a-single-image-and-a-collection-of-backgrounds-with-a-single-vec-file-as-an-output).

In [8]:
obj = 'bin'

## Create cut-out base images from *_info.dat

Which was created with `opencv_dat.ipynb`.

In [59]:
package_dirname = '/home/baxter/ros_ws/src/baxter_pick_and_place/'

positive_dirname = os.path.join(package_dirname, 'data', 'sdd')

base_dirname = os.path.join(package_dirname, 
                            'data', 'synthetic_demo_data')
out_dirname = os.path.join(base_dirname, 'base', obj)
annotations_dirname = os.path.join(base_dirname, obj)
                           
if not os.path.exists(out_dirname):
    os.makedirs(out_dirname)
if not os.path.exists(annotations_dirname):
    os.makedirs(annotations_dirname)

In [60]:
info_filename = os.path.join(positive_dirname, '%s_info.dat' % obj)

with open(info_filename, 'r') as fp:
    info = fp.read().splitlines()

for i in info:
    pth, _, roi = i.split('  ')
    img = cv2.imread(base_dirname + pth)
    roi = [int(a) for a in roi.split(' ')]
    x, y, w, h = roi
    img = img[y:y+h, x:x+w]
    cv2.imwrite(os.path.join(out_dirname, pth.split('/')[-1]), img)
print "Wrote %i cut-out images into %s." % (len(info), out_dirname)

Wrote 210 cut-out images into /home/baxter/ros_ws/src/baxter_pick_and_place/data/synthetic_demo_data/base/bin.


## Augment cut-out images

In [22]:
base_images = glob.glob(os.path.join(out_dirname, '*.jpg'))
print "Found %i cut-out samples." % len(base_images)

Found 210 cut-out samples.


In [48]:
num = 20
w = 20
h = 20

bg = os.path.join(base_dirname, 'bg.txt')

start = time.time()
for idx, base_image in enumerate(base_images):
    cmd = 'opencv_createsamples -img %s -bg %s -info %s -num %i -w %i -h %i' % \
        (base_image, 
         bg,
         os.path.join(annotations_dirname, 
                      os.path.join('%4i' % idx, 
                                   '%s.lst' % os.path.splitext(base_image)[0].split('/')[-1])), 
         num,
         w,
         h) + \
        '-maxxangle 0.1 -maxyangle 0.1 -maxzangle 0.1 -randinv -maxidev 255 -bgcolor 0 -bgthresh 0'
    call(cmd, shell=True)
print "Created %i samples from %i base images in %.2fs." % (num*len(base_images), 
                                                            len(base_images), 
                                                            time.time() - start)

Created 4200 samples from 210 base images in 64.87s.


## Combine annotations into *.vec

In [49]:
annotation_filenames = glob.glob(os.path.join(annotations_dirname, '*.lst'))
print "Found %i annotation files." % len(annotation_filenames)

Found 210 annotation files.


In [58]:
annotations_filename = os.path.join(annotations_dirname.split(obj)[0], '%s_info.dat' % obj)

with open(annotations_filename, 'w') as wfp:
    for af in annotation_filenames:
        with open(af, 'r') as rfp:
            for line in rfp:
                wfp.write('%s/%s' % (obj, line))
                
cmd = 'opencv_createsamples -vec %s -info %s -w %i -h %i -num %i' % \
    (os.path.join(annotations_dirname.split(obj)[0], '%s_%i_%i.vec' % (obj, w, h)), 
     annotations_filename, 
     w, 
     h, 
     num*len(annotation_filenames))
print '%s\n' % cmd
!{cmd}

opencv_createsamples -vec /home/baxter/ros_ws/src/baxter_pick_and_place/data/synthetic_demo_data/bin_20_20.vec -info /home/baxter/ros_ws/src/baxter_pick_and_place/data/synthetic_demo_data/bin_info.dat -w 20 -h 20 -num 4200

Info file name: /home/baxter/ros_ws/src/baxter_pick_and_place/data/synthetic_demo_data/bin_info.dat
Img file name: (NULL)
Vec file name: /home/baxter/ros_ws/src/baxter_pick_and_place/data/synthetic_demo_data/bin_20_20.vec
BG  file name: (NULL)
Num: 4200
BG color: 0
BG threshold: 80
Invert: FALSE
Max intensity deviation: 40
Max x angle: 1.1
Max y angle: 1.1
Max z angle: 0.5
Show samples: FALSE
Width: 20
Height: 20
Create training samples from images collection...
Done. Created 4200 samples
